<a href="https://colab.research.google.com/github/LatiefDataVisionary/multivariate-analysis-college-task/blob/main/src/DiscriminantAnalysis_week12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Import Libraries**

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import kagglehub
import os

Pada bagian ini, dilakukan impor terhadap pustaka-pustaka (libraries) Python yang esensial untuk analisis data, manipulasi data, pemodelan machine learning, dan visualisasi.

*   **Pandas**: Digunakan untuk manipulasi dan analisis data, khususnya untuk bekerja dengan struktur data DataFrame yang tabular.
*   **NumPy**: Merupakan pustaka fundamental untuk komputasi numerik di Python, menyediakan dukungan untuk array dan matriks multidimensi, beserta kumpulan fungsi matematika tingkat tinggi untuk beroperasi pada array tersebut.
*   **Scikit-learn (sklearn)**: Pustaka machine learning yang komprehensif. Dari sini, akan diimpor beberapa modul spesifik:
    *   `train_test_split`: Untuk membagi dataset menjadi set pelatihan dan pengujian.
    *   `LinearDiscriminantAnalysis`: Implementasi dari model Analisis Diskriminan Linier.
    *   `StandardScaler`: Untuk melakukan standarisasi fitur (opsional, tetapi seringkali bermanfaat).
    *   `accuracy_score`, `confusion_matrix`, `classification_report`: Metrik untuk mengevaluasi performa model klasifikasi.
*   **Matplotlib.pyplot** dan **Seaborn**: Digunakan untuk membuat visualisasi data statis, animasi, dan interaktif. (Meskipun mungkin tidak selalu digunakan secara ekstensif dalam LDA dasar, ada baiknya untuk diimpor jika diperlukan eksplorasi atau visualisasi hasil).
*   **KaggleHub** dan **OS**: Digunakan spesifik untuk mengunduh dan mengakses dataset dari Kaggle, sesuai dengan instruksi pemuatan data yang diberikan.

## **1. Load Dataset**

In [19]:
path = 'https://raw.githubusercontent.com/LatiefDataVisionary/multivariate-analysis-college-task/refs/heads/main/datasets/dataset_LDA_2clusters.csv'

df = pd.read_csv(path)
df.head()

,BloodPressure,Age,Cluster
0,72,50,0
1,66,31,1
2,64,32,1
3,66,21,1
4,40,33,1


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   BloodPressure  600 non-null    int64
 1   Age            600 non-null    int64
 2   Cluster        600 non-null    int64
dtypes: int64(3)
memory usage: 14.2 KB


In [21]:
df.describe()

,BloodPressure,Age,Cluster
count,600.000000,600.000000,600.000000
mean,72.041667,33.278333,0.651667
std,11.980086,11.822315,0.476840
min,24.000000,21.000000,0.000000
25%,64.000000,24.000000,0.000000
50%,72.000000,29.000000,1.000000
75%,80.000000,40.000000,1.000000
max,122.000000,81.000000,1.000000


In [22]:
df.shape

(600, 3)

## **2. Data Preprocessing and Preparation**

Tahap pra-pemrosesan data adalah langkah krusial sebelum membangun model machine learning. Tujuannya adalah untuk menyiapkan data dalam format yang sesuai untuk algoritma LDA.


1.  **Memisahkan Fitur (Variabel Independen) dan Target (Variabel Dependen)**:
    *   Variabel independen (fitur) adalah kolom-kolom yang akan digunakan untuk membuat prediksi. Dalam kasus ini, fitur adalah `BloodPressure` dan `Age`. Variabel-variabel ini akan disimpan dalam DataFrame `X`.
    *   Variabel dependen (target) adalah kolom yang ingin diprediksi. Dalam kasus ini, target adalah kolom `Cluster`. Variabel ini akan disimpan dalam Series `y`.
2.  **Membagi Data menjadi Set Pelatihan dan Pengujian (`train_test_split`)**:
    *   Dataset akan dibagi menjadi dua subset: set pelatihan (`X_train`, `y_train`) dan set pengujian (`X_test`, `y_test`).
    *   Model akan dilatih menggunakan set pelatihan.
    *   Performa model akan dievaluasi menggunakan set pengujian, yang merupakan data yang belum pernah dilihat oleh model selama proses pelatihan.
    *   Parameter `test_size=0.3` menunjukkan bahwa 30% dari data akan dialokasikan untuk set pengujian, dan sisanya (70%) untuk set pelatihan.
    *   Parameter `random_state=42` digunakan untuk memastikan bahwa pembagian data bersifat reproduktif. Artinya, jika kode dijalankan kembali dengan `random_state` yang sama, pembagian datanya akan selalu sama. Ini penting untuk konsistensi hasil.
3.  **Penskalaan Fitur (`StandardScaler`) (Opsional namun Direkomendasikan)**:
    *   LDA sensitif terhadap skala fitur, karena ia mencoba untuk memaksimalkan jarak antar kelas. Fitur dengan skala yang lebih besar mungkin mendominasi perhitungan jarak.
    *   `StandardScaler` dari scikit-learn digunakan untuk melakukan standarisasi fitur dengan menghilangkan mean dan menskalakan ke varians unit.
    *   Prosesnya adalah:
        *   Menginisialisasi objek `StandardScaler`.
        *   Melatih (fit) scaler **hanya** pada data fitur pelatihan (`X_train`). Ini penting untuk menghindari kebocoran data (data leakage) dari set pengujian ke proses pelatihan scaler.
        *   Mengaplikasikan (transform) penskalaan yang telah dipelajari ke data fitur pelatihan (`X_train`) dan data fitur pengujian (`X_test`).
    *   Setelah penskalaan, semua fitur akan memiliki mean sekitar 0 dan standar deviasi sekitar 1.

In [23]:
# Memastikan tidak ada nilai NaN di kolom yang akan digunakan
# Jika ada, perlu strategi penanganan (misal, imputasi atau penghapusan baris)
print("Missing values before any processing:")
print(df.isnull().sum())

Missing values before any processing:
BloodPressure    0
Age              0
Cluster          0
dtype: int64


In [24]:
# Jika ada NaN pada fitur atau target, dan ingin menghapus barisnya:
# df.dropna(subset=['BloodPressure', 'Age', 'Cluster'], inplace=True)
# print("\nMissing values after dropping NaN rows (if any):")
# print(df.isnull().sum())
# print("\nNew dataset shape after dropping NaN rows:", df.shape)

### **2.1. Memisahkan Fitur (X) dan Target (y)**

In [31]:
feature_columns = ['BloodPressure', 'Age']
target_column = 'Cluster'

X = df[feature_columns]
y = df[target_column]

print("\nShape of features (X):", X.shape)
print("Shape of target (y):", y.shape)
print("\nFirst 5 rows of features (X):")
print(X.head(10))
print("\nFirst 5 values of target (y):")
print(y.head(10))


Shape of features (X): (600, 2)
Shape of target (y): (600,)

First 5 rows of features (X):
   BloodPressure  Age
0             72   50
1             66   31
2             64   32
3             66   21
4             40   33
5             74   30
6             50   26
7             72   29
8             70   53
9             96   54

First 5 values of target (y):
0    0
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    0
9    0
Name: Cluster, dtype: int64


### **2.2. Membagi Data menjadi Set Pelatihan dan Pengujian**


In [33]:
# Umumnya menggunakan 70-80% untuk pelatihan dan 20-30% untuk pengujian
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
# stratify=y digunakan untuk memastikan proporsi kelas target sama di train dan test set, penting untuk klasifikasi

In [35]:
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (420, 2)
Shape of X_test: (180, 2)
Shape of y_train: (420,)
Shape of y_test: (180,)


In [36]:
print("\nProportion of target variable in original dataset:")
print(y.value_counts(normalize=True))
print("\nProportion of target variable in training set:")
print(y_train.value_counts(normalize=True))
print("\nProportion of target variable in test set:")
print(y_test.value_counts(normalize=True))


Proportion of target variable in original dataset:
Cluster
1    0.651667
0    0.348333
Name: proportion, dtype: float64

Proportion of target variable in training set:
Cluster
1    0.652381
0    0.347619
Name: proportion, dtype: float64

Proportion of target variable in test set:
Cluster
1    0.65
0    0.35
Name: proportion, dtype: float64


### **3. Penskalaan Fitur (Feature Scaling)**


Inisialisasi StandardScaler

In [32]:
# Inisialisasi StandardScaler
scaler = StandardScaler()

# Fit scaler pada data pelatihan dan transformasikan data pelatihan
X_train_scaled = scaler.fit_transform(X_train)

# Transformasikan data pengujian menggunakan scaler yang sudah di-fit pada data pelatihan
X_test_scaled = scaler.transform(X_test)

# Mengubah hasil scaling kembali menjadi DataFrame Pandas (opsional, untuk kemudahan inspeksi)
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=feature_columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=feature_columns)

print("First 5 rows of scaled training features (X_train_scaled_df):")
print(X_train_scaled_df.head())
print("\nDescriptive statistics of scaled training features:")
print(X_train_scaled_df.describe().round(2)) # .round(2) untuk membulatkan agar mudah dibaca
print("\nFirst 5 rows of scaled test features (X_test_scaled_df):")
print(X_test_scaled_df.head())
print("\n" + "="*50 + "\n")

NameError: name 'X_train' is not defined

**Poin Penting:**
- **Penanganan Missing value**:
  - Untuk dataset `dataset_LDA_2clusters.csv` yang diberikan, ternyata tidak ada nilai yang hilang, tetapi ini adalah praktik standar yang baik.
- **`stratify=y`**:
  - Saat melakukan `train_test_split`, `stratify=y` sangat penting untuk masalah klasifikasi, terutama jika distribusi kelas tidak seimbang. Ini memastikan bahwa proporsi setiap kelas dalam variabel target (y) dipertahankan baik di set pelatihan maupun di set pengujian.
- **Fitting Scaler**:
  - `StandardScaler` (atau scaler lainnya) selalu di-fit hanya pada data pelatihan (`X_train`) dan kemudian digunakan untuk mentransformasi baik data pelatihan (`X_train`) maupun data pengujian (`X_test`). Ini mencegah kebocoran informasi dari data pengujian ke dalam proses pelatihan model (atau pra-pemrosesan).

Setelah langkah-langkah ini, data `X_train_scaled`, `X_test_scaled`, `y_train`, dan `y_test` siap digunakan untuk membangun dan mengevaluasi model LDA.
